In [1]:
import pymupdf4llm

text = pymupdf4llm.to_markdown("kr-fsi-regulation.pdf")

Processing kr-fsi-regulation.pdf...
[                                        ] (0/3[=                                       ] ( 1/3[==                                      ] ( 2/34[===                                     ] ( 3/3[====                                    ] ( 4/34[=====                                   ] ( 5/3=[=======                                 ] ( 6/3[========                                ] ( 7/34[=========                               ] ( 8/3[==========                              ] ( 9/34[===========                             ] (10/3[============                            ] (11/34=[==============                          ] (12/34[===============                         ] (13/3[================                        ] (14/34[=================                       ] (15/3[==================                      ] (16/34=[====================                    ] (17/34[=====================                   ] (18/3[======================                  

In [2]:
import re


chapter_pattern = r"(?P<text>(?P<pre_chapter>.*?(?=\n제1장))|(?P<chapter>제(?P<num>\d+)장)\s*(?P<title>.*?)\n(?P<content>.*?)(?=\n(?:제\d+장|부칙)|\Z)|(?P<annex>\n부칙)\s*(?P<annex_content>.*))"

chapter_chunks = []
for match in re.finditer(chapter_pattern, text, re.DOTALL):
    if match.group('pre_chapter'):
        chapter_chunks.append({
            'type': 'PreChapter',
            'content': match.group('pre_chapter').strip(),
            'text': match.group('text').strip()
        })
    elif match.group('chapter'):
        chapter_chunks.append({
            'type': 'Chapter',
            'chapter': match.group('num'),
            'title': match.group('title'),
            'content': match.group('content').strip(),
            'text': match.group('text').strip()
        })
    elif match.group('annex'):
        chapter_chunks.append({
            'type': 'Annex',
            'chapter': '부칙',
            'title': match.group('annex'),
            'content': match.group('annex_content').strip(),
            'text': match.group('text').strip()
        })

#for chunk in chapter_chunks:
#    print(f"{chunk['content']}")


for chunk in chapter_chunks:
    if chunk['type'] == 'PreChapter':
        print("####" * 50)
        print("PreChapter:")
        print(chunk['content'][:50])
        print("####" * 50)
    elif chunk['type'] == 'Chapter':
        print("-" * 50)
        print(f"장 번호: {chunk['chapter']}, 장 제목: {chunk['title']}")
        print(f"장 내용: {chunk['content'][:50]}")
        print("-" * 50)
    else:
        print("===" * 50)
        print(f"부칙 제목: {chunk['title']}")
        print(f"부칙 내용: {chunk['content'][:50]}")
        print("===" * 50)



########################################################################################################################################################################################################
PreChapter:
# 전자금융감독규정

[시행 2023. 1. 1.] [금융위원회고시 제2022-44호, 2
########################################################################################################################################################################################################
--------------------------------------------------
장 번호: 1, 장 제목: 총칙
장 내용: 제2조(정의) 이 규정에서 사용하는 용어의 정의는 다음과 같다.

1. "전산실"이라 함은
--------------------------------------------------
--------------------------------------------------
장 번호: 2, 장 제목: 전자금융거래 당사자의 권리와 의무
장 내용: 제4조(확인에 필요한 구체적인 거래내용) 시행령 제7조제4항제6호에서 "금융위원회가 정하여
--------------------------------------------------
--------------------------------------------------
장 번호: 3, 장 제목: 전자금융거래의 안전성 확보 및 이용자 보호
장 내용: 제1절 통칙

제7조(전자금융거래 종류별 안전성 기준) 법 제21조제2항의 "금융위원회가 
------------------

In [3]:
# r"(?P<section>제(?P<num>\d+)절)\s*(?P<title>.*?)\n(?P<content>.*?)(?=\n(?:제\d+절|부칙)|\Z)",  # 절 패턴, 절 번호, 타이틀, 내용을 그룹으로 캡처
pattern = r"(?P<text>(?P<section>제(?P<num>\d+)절)[ ](?P<title>.*?)\n(?P<content>.*?)(?=\n?(?:제\d+절[ ]|$)))" #

section_chunks = []

for chapter in chapter_chunks:
    content = chapter['content']
        
    section_chunks.append(chapter)
    
    if re.search(r"(제1절)\s", content):
        for match in re.finditer(pattern, content, re.DOTALL):
            section = {}
            if chapter.get('chapter'):
                section['chapter'] = chapter.get('chapter')

            if match.group('section'):
                section['section'] = match.group('num').strip()
                section['title'] = match.group('title').strip()
                section['content'] = match.group('content').strip()
                section['text'] = match.group('text').strip()
                
                section_chunks.append(section)

for section in section_chunks:
    str = ""
    if section.get('chapter'):
        str += section['chapter'] + "장"
    if section.get('section'):
        str += section['section'] + "절"
        
    if section.get('title'):
        str += "(" + section['title'] + ")"
        
    if len(section['content']) > 30:
        str += " " + section['content'][:30].strip()
    else:
        str += " " + section['content'].strip()
        
    print(str)

 # 전자금융감독규정

[시행 2023. 1. 1.] [
1장(총칙) 제2조(정의) 이 규정에서 사용하는 용어의 정의는 다음
2장(전자금융거래 당사자의 권리와 의무) 제4조(확인에 필요한 구체적인 거래내용) 시행령 제7조
3장(전자금융거래의 안전성 확보 및 이용자 보호) 제1절 통칙

제7조(전자금융거래 종류별 안전성 기준)
3장1절(통칙) 제7조(전자금융거래 종류별 안전성 기준) 법 제21조제
3장2절(인력, 조직 및 예산 부문) 제8조(인력, 조직 및 예산) ① 금융회사 또는 전자금
3장3절(시설부문) 제9조(건물에 관한 사항) 금융회사 또는 전자금융업자는
3장4절(정보기술부문) 제12조(단말기 보호대책) 금융회사 또는 전자금융업자는
3장5절(정보기술부문 내부통제) 제19조(정보기술부문 계획서 제출 절차 등) ① 시행령
3장6절(전자금융업무) 제34조(전자금융거래 시 준수사항) 금융회사 또는 전자
4장(전자금융업의 허가와 등록 및 업무) 제1절 허가 및 등록의 대상과 절차

제42조(총발행잔
4장1절(허가 및 등록의 대상과 절차) 제42조(총발행잔액의 산정방법 등) ① 시행령 제15조
4장2절(허가 및 등록의 세부요건) 제50조(인력 및 물적 시설 세부요건) ① 법 제28조
4장3절(전자금융업의 업무) 제56조(전자화폐 발행업자의 겸업가능 업무) 시행령 제
5장(전자금융업무의 감독) 제58조(금융회사의 정보기술부문 실태평가 등) ① 금융
6장(보칙) 제73조(정보기술부문 및 전자금융 사고보고) ① 금융회
부칙장(
부칙) <제2022-44호,2022.11.23.>

제1조(시


In [4]:
pattern = r"(?P<text>(?P<article>제(?P<num>\d+)조(?:의(?P<subnum>\d+))?)\((?P<title>[^)]*)\)\s*(?P<content>.*?)(?=\n?(?:제\d+조(?:의\d+)?)|$))"  # 조 패턴, 조 번호, 제목, 내용을 그룹으로 캡처

article_chunks = []

for section in section_chunks:
    content = section['content']
    
    for match in re.finditer(pattern, content, re.DOTALL):
        article = {}
        if section.get('chapter'):
            article['chapter'] = section.get('chapter')

        if section.get('section'):
            article['section'] = section.get('section')
        
        if match.group('article'):
            article['article'] = match.group('num').strip()
            
            if match.group('subnum'):
                article['sub-article'] = match.group('subnum').strip()

            article['title'] = match.group('title').strip()
            article['content'] = match.group('content').strip()
            article['text'] = match.group('text').strip()
            
            article_chunks.append(article)

for article in article_chunks:
    str = ""
    if article.get('chapter'):
        str += article['chapter'] + "장"
    if article.get('section'):
        str += article['section'] + "절 "
        
    if article.get('article'):
        str += article['article'] + "조"
        if article.get('sub-article'):
            str += "의" + article['sub-article']
    if article.get('title'):
        str += "(" + article['title'] + ") "
        
    if len(article['content']) > 30:
        str += " " + article['content'][:30].strip()
    else:
        str += " " + article['content'].strip()
        
    print(str)

1조(목적)  이 규정은 「전자금융거래법」(이하 "법"이라 한다) 및
1장2조(정의)  이 규정에서 사용하는 용어의 정의는 다음과 같다.

1
1장3조(전자금융보조업자의 범위)  법
2장4조(확인에 필요한 구체적인 거래내용)  시행령
2장5조(전자금융사고 책임이행을 위한 보험 등의 가입에 관한 기준)  ① 금융회사 또는 전자금융업자가 법
2장6조(추심이체 출금 동의의 방법 등)  ① 시행령
2장6조의2(정보보호최고책임자의 지정대상)  ① 시행령
3장7조(전자금융거래 종류별 안전성 기준)  법
3장8조(인력, 조직 및 예산)  ① 금융회사 또는 전자금융업자는 인력 및 조직의 운용에
3장8조의2(정보보호위원회 운영)  ① 금융회사 또는 전자금융업자는 중요 정보보호에 관한
3장9조(건물에 관한 사항)  금융회사 또는 전자금융업자는 전산실이 위치한 건물에 관
3장10조(전원, 공조 등 설비에 관한 사항)  금융회사 또는 전자금융업자는 전산실이 위치한 건물의 전
3장11조(전산실 등에 관한 사항)  금융회사 또는 전자금융업자는 전산실에 관하여 다음 각
3장12조(단말기 보호대책)  금융회사 또는 전자금융업자는 단말기 보호를 위하여 다음
3장13조(전산자료 보호대책)  ① 금융회사 또는 전자금융업자는 전산자료의 유출, 파괴
3장14조(정보처리시스템 보호대책)  금융회사 또는 전자금융업자는 정보처리시스템의 안전한 운
3장14조의2(클라우드컴퓨팅서비스 이용절차 등)  ① 금융회사 또는 전자금융업자는 「클라우드컴퓨팅 발전
3장15조(해킹 등 방지대책)  ① 금융회사 또는 전자금융업자는 정보처리시스템 및 정보
3장16조(악성코드 감염 방지대책)  ① 금융회사 또는 전자금융업자는 악성코드 감염을 방지하
3장17조(홈페이지 등 공개용 웹서버 관리대책)  ① 금융회사 또는 전자금융업자는 공개용 웹서버의 안전한
3장18조(IP주소 관리대책)  금융회사 또는 전자금융업자는 정보제공자 주소(이하 "I
3장19조(정보기술부문 계획서 제출 절차 등)  ① 시행령
3장19조의2(정보보호 교육계

In [5]:
import unicodedata

pattern = r"\n?(?P<text>(?P<paragraph>[\u2460-\u24EA])\s*(?P<content>.*?)(?=(?:\n?[\u2460-\u24EA]\s*|$)))"  # 항 패턴, 항 번호, 내용을 그룹으로 캡처 

paragraph_chunks = []

for article in article_chunks:
    content = article['content']
    
    if content.startswith("금융위원회는 법"):
        print(content)
    
    if re.search(r"([\u2460-\u24EA])\s*", content):           
        for match in re.finditer(pattern, content, re.DOTALL):
            paragraph = {}
            
            if article.get('chapter') is not None:
                paragraph['chapter'] = article.get('chapter')
            if article.get('section') is not None:
                paragraph['section'] = article.get('section')
            if article.get('article') is not None:
                paragraph['article'] = article.get('article')
            if article.get('sub-article') is not None:
                paragraph['sub-article'] = article.get('sub-article')

            if match.group('paragraph'):
                paragraph['paragraph'] = f"{int(unicodedata.numeric(match.group('paragraph')))}"
                paragraph['content'] = match.group('content').strip()
                paragraph['text'] = match.group('text').strip()
                
                paragraph_chunks.append(paragraph)
    else:
        paragraph_chunks.append(article)


for paragraph in paragraph_chunks:
    str = ""
    if paragraph.get('chapter'):
        str += paragraph['chapter'] + "장"
    else:
        str += "  "
    if paragraph.get('section'):
        str += paragraph['section'] + "절"
        
    if paragraph.get('article'):
        str += paragraph['article'] + "조"
        if paragraph.get('sub-article'):
            str += "의" + paragraph['sub-article']
        
    if paragraph.get('paragraph'):
        str += "(" + paragraph['paragraph'] + "항)"
        
    if len(paragraph['content']) > 20:
        str += " " + paragraph['content'][:20].strip()
    else:
        str += " " + paragraph['content'].strip()
        
    print(str)


  1조 이 규정은 「전자금융거래법」(이하 "
1장2조 이 규정에서 사용하는 용어의 정의는
1장3조 법
2장4조 시행령
2장5조(1항) 금융회사 또는 전자금융업자가 법
2장6조(1항) 시행령
2장6조의2(1항) 시행령
3장7조 법
3장8조(1항) 금융회사 또는 전자금융업자는 인력 및
3장8조(2항) 금융회사 또는 전자금융업자는 인력 및
3장8조(3항) 제2항 각 호의 사항을 이행하지 못하
3장8조(4항) 제2항제1호의 인력에 관한 기준은 <
3장8조의2(1항) 금융회사 또는 전자금융업자는 중요 정
3장8조의2(2항) 정보보호위원회의 장은 정보보호최고책임
3장8조의2(3항) 정보보호위원회는 다음 각 호의 사항을
3장9조 금융회사 또는 전자금융업자는 전산실이
3장10조 금융회사 또는 전자금융업자는 전산실이
3장11조 금융회사 또는 전자금융업자는 전산실에
3장12조 금융회사 또는 전자금융업자는 단말기
3장13조(1항) 금융회사 또는 전자금융업자는 전산자료
3장13조(2항) 제1항제1호의 사용자계정의 공동 사용
3장13조(3항) 금융회사 또는 전자금융업자는 단말기를
3장13조(4항) 제1항제11호의 정보처리시스템 가동기
3장13조(5항) 금융회사 또는 전자금융업자는 단말기와
3장14조 금융회사 또는 전자금융업자는 정보처리
3장14조의2(1항) 금융회사 또는 전자금융업자는 「클라우
3장15조(1항) 금융회사 또는 전자금융업자는 정보처리
3장15조(2항) 제1항제1호의 규정에 따른 정보보호시
3장15조(3항) 제1항 각 호의 정보보호시스템에 대하
3장15조(4항) 금융회사 또는 전자금융업자는 해킹 등
3장15조(5항) 삭제<2013. 12. 3.>
3장15조(6항) 금융회사 또는 전자금융업자는 무선통신
3장16조(1항) 금융회사 또는 전자금융업자는 악성코드
3장17조(1항) 금융회사 또는 전자금융업자는 공개용
3장17조(2항) 금융회사 또는 전자금융업자는 공개용
3장17조(3항) 삭제<2013. 12. 3.>
3장17조(4항) 금융회사 또는 전자금융업자는 공개용
3장17

In [11]:
pattern = r"\n(?P<text>(?P<clause>\d+)\.\s*(?P<content>.*?)(?=\n?\d+\.\s(?!\d+\.\s\d+\.)|$))" # 호 패턴

clause_chunks = []

for paragraph in paragraph_chunks:
    content = paragraph['content']
    
    content = re.sub(r'<.*?\n*.*?>', lambda match: match.group().replace('\n', ''), content)
    
    if re.search(r"\n?(1.)\s", content):
        for match in re.finditer(pattern, content, re.DOTALL):
            clause = {}
            if paragraph.get('chapter') is not None:
                clause['chapter'] = paragraph.get('chapter')
            if paragraph.get('section') is not None:
                clause['section'] = paragraph.get('section')
            if paragraph.get('article') is not None:
                clause['article'] = paragraph.get('article')
            if paragraph.get('sub-article') is not None:
                clause['sub-article'] = paragraph.get('sub-article')
            if paragraph.get('paragraph'):
                clause['paragraph'] = paragraph.get('paragraph')

            if match.group('clause'):
                clause['clause'] = match.group('clause')
                clause['content'] = match.group('content').strip()
                clause['text'] = match.group('text').strip()
                
                clause_chunks.append(clause)
    else:
        clause_chunks.append(paragraph)

for clause in clause_chunks:
    
    code = ""
    if clause.get('chapter'):
        if clause['chapter'] == "부칙":
            code += "부칙"
        else:
            code += "제" + clause['chapter'] + "장"
    if clause.get('section'):
        code += "제" + clause['section'] + "절"
        
    if clause.get('article'):
        code += "제" + clause['article'] + "조"
        if clause.get('sub-article'):
            code += "의" + clause['sub-article']
        
    if paragraph.get('paragraph'):
        code += "제" + paragraph['paragraph'] + "항"
        
    if clause.get('clause'):
        code += "제" + clause['clause'] + "호"
        
    str = ""
    if len(clause['content']) > 20:
       str += " " + clause['content'][:20].strip()
    else:
        str += " " + clause['content'].strip()
        
    clause['code'] = code
        
    print(f"{code} {str}")

제1조  이 규정은 「전자금융거래법」(이하 "
제1장제2조제1호  "전산실"이라 함은 전산장비, 통신
제1장제2조제2호  "전산자료"라 함은 전산장비에 의해
제1장제2조제3호  "정보처리시스템"이라 함은 전자금융업
제1장제2조제4호  "정보기술부문"이라 함은 컴퓨터 등
제1장제2조제5호  "정보보호" 또는 "정보보안"이라 함
제1장제2조제6호  "정보보호시스템"이라 함은 정보처리시
제1장제2조제7호  "해킹"이라 함은 접근을 허가받지 아
제1장제2조제8호  "컴퓨터악성코드"(이하 "악성코드"라
제1장제2조제9호  "공개용 웹서버"라 함은 인터넷 이용
제1장제2조제10호  "정보통신망"(이하 "통신망"이라 한
제1장제2조제11호  삭제<2013.
제1장제3조  법
제2장제4조  시행령
제2장제5조  금융회사 또는 전자금융업자가 법
제2장제6조  시행령
제2장제6조의2  시행령
제3장제7조  법
제3장제8조제1호  정보처리시스템 및 전자금융업무 관련
제3장제8조제2호  외부주문등에 관한 계약을 체결하는 때
제3장제8조제3호  전산인력의 자질향상 및 예비요원 양성
제3장제8조제4호  정보보호최고책임자는 임직원이 정보보안
제3장제8조제5호  최고경영자는 임직원이 정보보안 관련법
제3장제8조제1호  정보기술부문 인력은 총 임직원수의 1
제3장제8조제2호  정보보호예산을 정보기술부문 예산의 1
제3장제8조  제2항제1호의 인력에 관한 기준은 <
제3장제8조의2  금융회사 또는 전자금융업자는 중요 정
제3장제8조의2  정보보호위원회의 장은 정보보호최고책임
제3장제8조의2제1호  법
제3장제9조제1호  건물 출입구는 경비원에 의하여 통제하
제3장제9조제2호  비상시 대피를 위한 비상계단 및 정전
제3장제9조제3호  번개, 과전류 등 고전압으로 인한 전
제3장제9조제4호  서버, 스토리지(Storage) 등
제3장제9조제5호  화재발생 시 조기진압을 위한 소화기
제3장제9조제6호  화재발생 위험이 높은 지역, 상습 침
제3장제10조제1호  전원실, 공조실 등 주요 설비시설에


In [12]:
pattern = r"(?P<text>(?P<item>[가나다라마바사아자차카타파하]?)\.\s*(?P<content>.*?)(?=\n?[가나다라마바사아자차카타파하]?\.\s*.*|$))" # 목 패턴 (?![가나다라마바사아자차카타파하]\.\n)

item_chunks = []

for clause in clause_chunks:
    content = clause['content']
    
    if re.search(r"\n(가.)\s", content):
        for match in re.finditer(pattern, content, re.DOTALL):
            item = {}
            if clause.get('chapter') is not None:
                item['chapter'] = clause.get('chapter')
            if clause.get('section') is not None:
                item['section'] = clause.get('section')
            if clause.get('article') is not None:
                item['article'] = clause.get('article')
            if clause.get('sub-article') is not None:
                item['sub-article'] = clause.get('sub-article')
            if clause.get('paragraph'):
                item['paragraph'] = clause.get('paragraph')
            if clause.get('clause'):
                item['clause'] = clause.get('clause')

            if match.group('item'):
                item['item'] = match.group('item')
                item['content'] = match.group('content').strip()
                item['text'] = match.group('text').strip()

                item_chunks.append(item)
    else:
        item_chunks.append(clause)
        

for item in item_chunks:
    
    code = ""
    if item.get('chapter'):
        if item['chapter'] == "부칙":
            code += "부칙"
        else:
            code += "제" + item['chapter'] + "장"
    if item.get('section'):
        code += "제" + item['section'] + "절"
        
    if item.get('article'):
        code += "제" + item['article'] + "조"
        if item.get('sub-article'):
            code += "의" + item['sub-article']
        
    if item.get('paragraph'):
        code += "제" + item['paragraph'] + "항"
        
    if item.get('clause'):
        code += "제" + item['clause'] + "호"
    if item.get('item'):
        str += "제" + item['item'] + "목"
        
    str = ""
    if len(item['content']) > 20:
       str += " " + item['content'][:20].strip()
    else:
        str += " " + item['content'].strip()
        
    item['code'] = code
        
    print(f"{code} {str}")

제1조  이 규정은 「전자금융거래법」(이하 "
제1장제2조제1호  "전산실"이라 함은 전산장비, 통신
제1장제2조제2호  "전산자료"라 함은 전산장비에 의해
제1장제2조제3호  "정보처리시스템"이라 함은 전자금융업
제1장제2조제4호  "정보기술부문"이라 함은 컴퓨터 등
제1장제2조제5호  "정보보호" 또는 "정보보안"이라 함
제1장제2조제6호  "정보보호시스템"이라 함은 정보처리시
제1장제2조제7호  "해킹"이라 함은 접근을 허가받지 아
제1장제2조제8호  "컴퓨터악성코드"(이하 "악성코드"라
제1장제2조제9호  "공개용 웹서버"라 함은 인터넷 이용
제1장제2조제10호  "정보통신망"(이하 "통신망"이라 한
제1장제2조제11호  삭제<2013.
제1장제3조  법
제2장제4조  시행령
제2장제5조제1항  금융회사 또는 전자금융업자가 법
제2장제6조제1항  시행령
제2장제6조의2제1항  시행령
제3장제7조  법
제3장제8조제1항제1호  정보처리시스템 및 전자금융업무 관련
제3장제8조제1항제2호  외부주문등에 관한 계약을 체결하는 때
제3장제8조제1항제3호  전산인력의 자질향상 및 예비요원 양성
제3장제8조제1항제4호  정보보호최고책임자는 임직원이 정보보안
제3장제8조제1항제5호  최고경영자는 임직원이 정보보안 관련법
제3장제8조제2항제1호  정보기술부문 인력은 총 임직원수의 1
제3장제8조제2항제2호  정보보호예산을 정보기술부문 예산의 1
제3장제8조제4항  제2항제1호의 인력에 관한 기준은 <
제3장제8조의2제1항  금융회사 또는 전자금융업자는 중요 정
제3장제8조의2제2항  정보보호위원회의 장은 정보보호최고책임
제3장제8조의2제3항제1호  법
제3장제9조제1호  건물 출입구는 경비원에 의하여 통제하
제3장제9조제2호  비상시 대피를 위한 비상계단 및 정전
제3장제9조제3호  번개, 과전류 등 고전압으로 인한 전
제3장제9조제4호  서버, 스토리지(Storage) 등
제3장제9조제5호  화재발생 시 조기진압을 위한 소화기
제3장제9조제6호  화재발생 위험이 높은

In [14]:
import json
with open('fsi-regulation.json', 'w') as f:
    json.dump(item_chunks, f)

In [13]:
import csv
from collections import OrderedDict

sorted_field = ["code", "chapter", "section", "paragraph", "article", "sub-article", "clause", "item"]

with open('fsi-regulation.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    
    # 모든 키를 포함하는 헤더 생성
    fieldnames = OrderedDict()
    for item in item_chunks:
        for key in item.keys():
            fieldnames[key] = None
            
    fieldnames = list(fieldnames.keys())
    # 키 순서 변경
    fieldnames = sorted_field + [key for key in fieldnames if key not in sorted_field]
    
    writer.writerow(fieldnames)
    
    # JSON 데이터의 각 요소를 CSV 파일에 행으로 작성
    for item in item_chunks:
        row = [item.get(field, '') for field in fieldnames]
        writer.writerow(row)

